In [1]:
import openslide
import numpy as np
import scipy.misc
import scipy.io
import cv2
import os
import matplotlib.pyplot as plt    
from skimage import measure

In [2]:
# 获取切片单元的上下y值
def RemoveHeadEnd(OriImg):
    n, m, z = OriImg.shape
    i = 0
    while np.sum(OriImg[i, :, :]) == 0 and i<n-1:
        i += 1
    h = i
    j = n - 1
    while np.sum(OriImg[j, :, :]) == 0 and j>0:
        j -= 1
    e = j
    return h, e

In [ ]:
def get_result(img):
    bw = np.where(np.logical_and(img[:, :, 0] > 128, np.logical_or(img[:, :, 0] - img[:, :, 1] > 30,
                                                                   img[:, :, 0] - img[:, :, 2] > 30)), 1, 0)
    bw = bw.astype(dtype=np.float32)
    img_Blur = cv2.blur(bw, (100, 100), cv2.BORDER_REPLICATE)
    result = img_Blur
    result[result > 0.4] = 1
    result[result <= 0.4] = 0
    print('result == ', np.sum(result))
    return result

In [ ]:
if not os.path.exists('./SegPic'):
    os.mkdir('./SegPic')
svsFiles_dir = '/home/frodo/Desktop/job content/SVSFiles/'
MaskPath = '/home/frodo/Desktop/job content/所有mask/'
svsFiles_names = os.listdir(svsFiles_dir)

In [ ]:
for imgname in svsFiles_names:
    loc_dim = []
    imgname = '8647.svs'
    slide = openslide.open_slide(svsFiles_dir + imgname)
    # 获得要切片的图像大小
    size = slide.level_dimensions[1]
    # 获得对应图像大小下的缩放倍率
    zoom_ratio = int(slide.level_downsamples[1])
    # print(size)
    img = np.array(slide.read_region((0, 0), 1, size), dtype=np.float16)
    n, m, z = img.shape

    # 获得重要的信息点
    result = get_result(img)
    # 生成一个mask，并给mask的每一通道赋予相同的result值
    bw3 = np.zeros((n, m, z - 1))
    for i in range(z-1):
        bw3[:, :, i] = result
    # 获得去除背景后的图片
    BackRemImg = bw3 * img[:, :, :-1]

    # 生成（1,m）的数组，为图片的宽（x轴）
    PatchSegMark = np.zeros((1, m))
    # 如果图片的每一单位像素的行的值的总和不为0,说明有切片单元，值设为1；反之，说明没有切片单元，则该位置值设为0
    for i in range(m):
        if np.sum(result[:, i]):
            PatchSegMark[0, i] = 1
        else:
            PatchSegMark[0, i] = 0
    # 对相邻的值做差离值，若值为1（1-0）或者-1（0-1），说明这是切片单元的边界处
    PatchSegPos = np.diff(PatchSegMark, n=1)
    # 值为1代表切片单元的起始边界
    PatchSegPosStart = np.where(PatchSegPos == 1)[1]
    # 值为-1代表切片单元的结束边界
    PatchSegPosEnd = np.where(PatchSegPos == -1)[1]
    # 获得起始边界和结束边界的个数
    StartNum = len(PatchSegPosStart)
    EndNum = len(PatchSegPosEnd)
        
    # 确定切片单元的位置
    if PatchSegPosStart[0] > PatchSegPosEnd[0]:         # 代表先出现切片单元
        patch = BackRemImg[:, :PatchSegPosEnd[0], :]    # 第一张图片在0～第一个切片单元的结束边界选取
        h, e = RemoveHeadEnd(patch)                     # 获得每个切片单元的上下边界
        loc_dim.append(((0 * zoom_ratio, h * zoom_ratio),((PatchSegPosEnd[0]) * zoom_ratio, (e-h) * zoom_ratio)))

        if StartNum < EndNum:
            for i in range(StartNum):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i + 1]+1, :]
                h, e = RemoveHeadEnd(patch)
                if e > h:
                    loc_dim.append(((PatchSegPosStart[i] *zoom_ratio, h*zoom_ratio), ((PatchSegPosEnd[i + 1] - PatchSegPosStart[i])*zoom_ratio, (e-h)*zoom_ratio)))
        else:
            for i in range(StartNum - 1):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i + 1]+1, :]
                h, e = RemoveHeadEnd(patch)
                loc_dim.append(((PatchSegPosStart[i]*zoom_ratio,h*zoom_ratio),((PatchSegPosEnd[i + 1] -PatchSegPosStart[i])*zoom_ratio, (e-h)*zoom_ratio)))

            patch = BackRemImg[:, PatchSegPosStart[StartNum - 1]:m+1, :]
            h, e = RemoveHeadEnd(patch)
            loc_dim.append(((PatchSegPosStart[StartNum - 1]*zoom_ratio,h*zoom_ratio),((m-PatchSegPosStart[StartNum - 1])*zoom_ratio,(e-h)*zoom_ratio)))
    else:
        if StartNum > EndNum:
            for i in range(StartNum - 1):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i]+1, :]
                h, e = RemoveHeadEnd(patch)
                loc_dim.append(((PatchSegPosStart[i]*zoom_ratio,h*zoom_ratio),((PatchSegPosEnd[i] -PatchSegPosStart[i])*zoom_ratio, (e-h)*zoom_ratio)))
            patch = BackRemImg[:, PatchSegPosStart[StartNum - 1]:m+1, :]
            h, e = RemoveHeadEnd(patch)
            loc_dim.append(((PatchSegPosStart[StartNum - 1]*zoom_ratio,h*zoom_ratio),((m-PatchSegPosStart[StartNum - 1])*zoom_ratio,(e-h)*zoom_ratio)))
        else:
            print(EndNum)
            for i in range(StartNum):
                patch = BackRemImg[:, PatchSegPosStart[i]:PatchSegPosEnd[i]+1, :]
                h, e = RemoveHeadEnd(patch)
                loc_dim.append(((PatchSegPosStart[i]*zoom_ratio,h*zoom_ratio),((PatchSegPosEnd[i] -PatchSegPosStart[i])*zoom_ratio,(e-h)*zoom_ratio)))

    # 根据低分辨率的切片单元的位置，判定出高分辨率下对应切片单元的位置，进行切片
    print(imgname, ' started!')
    for each in loc_dim:
        i = loc_dim.index(each) + 1
        
        # each[0]为坐标位置，each[1]为图片宽高
        img = np.array(slide.read_region(each[0], 0, each[1]))
        scipy.misc.imsave('{}.jpg'.format(i), img[:,:,:-1])
        img_mask_path = MaskPath + imgname + '_' + str(i)+'.mask.png.bmp'
        # 读取二值化图片
        bmp = cv2.imread(img_mask_path, 0)
        # 判断是否有图片
        if bmp is not None:
            # 先创建对应图片的文件夹
            if not os.path.exists('./SegPic/' + imgname):
                os.mkdir('./SegPic/' + imgname)
            
            # 为当前图片的第i个切片单元创建对应的文件夹,如果已经存在，则直接处理下一个切片单元
            unit_path = 'SegPic/'+imgname+'/'+str(i)      # 当前切片单元路径
            if os.path.exists(unit_path):
                continue
            os.mkdir(unit_path)
            
            bw = np.where(bmp, 1, 0)        # 生成阈值
            label_image =measure.label(bw)  # 连通区域标记
            # 获得每一个连通域
            for region in measure.regionprops(label_image):
                pos = region.bbox
                if pos[2]-pos[0]<10 or pos[3]-pos[1]<10:
                    continue
                else:
                    result = np.zeros(((pos[2]-pos[0])*zoom_ratio, (pos[3]-pos[1])*zoom_ratio, 3), dtype=np.uint8)
                    for j in range(3):
                        result[:,:,j] = img[pos[0]*zoom_ratio:pos[2]*zoom_ratio, pos[1]*zoom_ratio:pos[3]*zoom_ratio, j]
                    scipy.misc.imsave(unit_path+'/{}.tif'.format(measure.regionprops(label_image).index(region)+1), result)
#         break
    print(imgname, ' finished! \n')
    break
